In [68]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/diploma/data.csv


In [69]:
# импорт библиотек

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling
from pandas_profiling import ProfileReport
import os
import sys
import zipfile
import PIL
from PIL import ImageOps, ImageFilter
import re
import ast
from datetime import time, date, datetime
import random

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-paper')

#from optuna import *

from sklearn import preprocessing
#from sklearn.feature_extraction import DictVectorizer
#from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler
#from sklearn.impute import KNNImputer

#from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import Ridge
#from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score
from sklearn import metrics
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor, Pool, cv

#image augmentation
#import albumentations as A
#from ImageDataAugmentor.image_data_augmentor import *

# keras
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#from tensorflow.keras.applications.efficientnet import EfficientNetB3
#from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import MeanAbsolutePercentageError

#from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, DataCollatorWithPadding

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', 550) # больше колонок

os.environ['PYTHONHASHSEED']=str(42)

In [70]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
ITERATIONS = 200

In [71]:
DIR_DATA  = '../input/diploma/'

In [95]:
data_d = pd.read_csv(DIR_DATA+'data.csv')

In [73]:
def facts_extraction(slovar):
    if slovar.find('atAGlanceFacts')>-1:
        slovar1 = eval(slovar)
       # print(type(slovar1))
        #print(slovar1)
        slovar2=slovar1.get('atAGlanceFacts')
       # print(type(slovar2))
       # print(slovar2)
        
        lst = ['']
        for i in slovar2:
            
            lst.append(i.get('factValue'))
        lst_res=lst[1:]
        return lst_res
            
    else: return np.NaN

In [103]:
r = "{'atAGlanceFacts': [{'factValue': '2008', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Forced Air', 'factLabel': 'Heating'}, {'factValue': 'Central', 'factLabel': 'Cooling'}, {'factValue': 'Attached Garage', 'factLabel': 'Parking'}, {'factValue': '', 'factLabel': 'lotsize'}, {'factValue': '$171/sqft', 'factLabel': 'Price/sqft'}]}"
print(type(r))

<class 'str'>


In [104]:
facts_extraction(r)

['2008', '', 'Forced Air', 'Central', 'Attached Garage', '', '$171/sqft']

In [105]:
type(facts_extraction(r))

list

In [96]:
data_d.homeFacts = data_d.homeFacts.apply(lambda x: facts_extraction(x))

In [97]:
data_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [98]:
data_d.head(4)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"[2019, , Central A/C, Heat Pump, , , None, $144]",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"[2019, , , , , 5828 sqft, $159/sqft]",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"[1961, 1967, Forced Air, Central, Attached Gar...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"[2006, 2006, Forced Air, Central, Detached Gar...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"


In [110]:
t = np.array(data_d['homeFacts'])
t

array([list(['2019', '', 'Central A/C, Heat Pump', '', '', None, '$144']),
       list(['2019', '', '', '', '', '5828 sqft', '$159/sqft']),
       list(['1961', '1967', 'Forced Air', 'Central', 'Attached Garage', '8,626 sqft', '$965/sqft']),
       ...,
       list(['1950', '1950', 'Other', '', '2', '1,600 sqft', '$458/sqft']),
       list([None, None, None, None, None, None, None]),
       list(['2019', None, 'Electric', 'Central', 'No Data', '6,969 sqft', '$140'])],
      dtype=object)

In [108]:
t1 = pd.DataFrame(t)

In [109]:
t1

,0
0,"[2019, , Central A/C, Heat Pump, , , None, $144]"
1,"[2019, , , , , 5828 sqft, $159/sqft]"
2,"[1961, 1967, Forced Air, Central, Attached Gar..."
3,"[2006, 2006, Forced Air, Central, Detached Gar..."
4,"[, , , , , 10,019 sqft, None]"
...,...
377180,"[1990, 1990, Other, Central, 2 spaces, 8,500 s..."
377181,"[1924, , Radiant, , None, , $337/sqft]"
377182,"[1950, 1950, Other, , 2, 1,600 sqft, $458/sqft]"
377183,"[None, None, None, None, None, None, None]"


In [ ]:
# data_d_1 = pd.concat([data_d[['private pool','propertyType','street','baths','homeFacts',
                             'fireplace','city','schools','sqft','zipcode','beds','state','stories','mls-id','PrivatePool','MlsId','target']],
                      data_d['homeFacts'].str.split(', ', expand=True)], axis=1)